In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\boston.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
x_train, x_test = train_test_split(df, test_size=0.2,random_state=42)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
tfd = tfp.distributions

def make_prior():
    return tfd.Normal(loc=0., scale=1.)

def make_posterior(latent_dim):
    return lambda: tfd.Normal(
        loc=tf.Variable(tf.random.normal([latent_dim])),
        scale=tf.nn.softplus(tf.Variable(tf.random.normal([latent_dim])))
    )

def create_bayesian_autoencoder(input_dim=784, latent_dim=16):
    encoder = tf.keras.Sequential([
        tfp.layers.DenseVariational(
            units=64, make_prior_fn=make_prior, make_posterior_fn=lambda: make_posterior(64), activation="relu"
        ),
        tfp.layers.DenseVariational(
            units=latent_dim, make_prior_fn=make_prior, make_posterior_fn=lambda: make_posterior(latent_dim)
        )
    ])
    
    decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(input_dim, activation="sigmoid")
    ])
    
    model = tf.keras.Model(inputs=tf.keras.Input(shape=(input_dim,)), outputs=decoder(encoder.output))
    
    model.compile(optimizer="adam", loss="mse")
    return model, encoder, decoder

In [ ]:
autoencoder, encoder, decoder = create_bayesian_autoencoder()

history = autoencoder.fit(x_train, x_train, epochs=20, batch_size=256, validation_split=0.2, verbose=1)

In [ ]:
x_test_pred = autoencoder.predict(x_test)

In [ ]:
n = 10 
plt.figure(figsize=(10, 4))
for i in range(n):
    plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap="gray")
    plt.axis("off")
    
    plt.subplot(2, n, i + 1 + n)
    plt.imshow(x_test_pred[i].reshape(28, 28), cmap="gray")
    plt.axis("off")

plt.suptitle("Original vs Reconstructed Images (Bayesian Autoencoder)")
plt.show()

In [ ]:
reconstruction_errors = np.mean(np.abs(x_test - x_test_pred), axis=1)

plt.figure(figsize=(6, 4))
plt.hist(reconstruction_errors, bins=50, color="blue", alpha=0.7)
plt.xlabel("Reconstruction Error")
plt.ylabel("Frequency")
plt.title("Reconstruction Error Distribution")
plt.show()

threshold = np.percentile(reconstruction_errors, 95) 
anomalies = np.where(reconstruction_errors > threshold)[0]

print(f"🔹 Jumlah Anomali Terdeteksi: {len(anomalies)}")